In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.decomposition import PCA

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import optuna

# Data Loading

In [2]:
train = pd.read_csv('train.csv', index_col='id')
original = pd.read_csv('original.xls', index_col='Row#')
test = pd.read_csv('test.csv', index_col='id')
ss = pd.read_csv('sample_submission.csv')

# Combine Train and Test Data

In [3]:
train1 = pd.concat([train, original]).reset_index(drop=True)

In [4]:
train2 = train1.drop('yield', axis=1).copy()
train2 = pd.concat([train2, test])

# Feature Enginering

In [5]:
# train2 = train1.copy()

In [6]:
# pca = PCA(n_components=3)
# transformed = pca.fit_transform(train2)
# new_features_df = pd.DataFrame(transformed, columns=['pca_feature_1', 'pca_feature_2', 'pca_feature3'])
# train2 = pd.concat([new_features_df, train2], axis=1)

# Feature Scaling

In [7]:
train3 = train2.copy()

In [8]:
sc = StandardScaler()
train3 = pd.DataFrame(sc.fit_transform(train3), columns=train2.columns)

# Split Train and Test data, transform

In [9]:
X = train3.loc[:train1.index.max(), :].copy()
test_transformed = train3.iloc[train1.index.max()+1:, :]

y = train1['yield']

# Feature Selection

In [12]:
# Define the linear regression model
lr = LinearRegression()

# Define the initial set of features to consider
features_lr = X.columns.to_list()

# Define the minimum MAE threshold
min_mae = np.inf

# Iterate over the features and remove the one that results in the lowest MAE
while len(features_lr) > 1:
    mae_values = []
    for feature in features_lr:
        temp_features = features_lr.copy()
        temp_features.remove(feature)
        lr.fit(X[temp_features], y)
        y_pred = lr.predict(X[temp_features])
        mae = mean_absolute_error(y, y_pred)
        mae_values.append(mae)
    best_feature_index = np.argmin(mae_values)
    best_feature_mae = mae_values[best_feature_index]
    if best_feature_mae < min_mae:
        min_mae = best_feature_mae
        features_lr.pop(best_feature_index)
    else:
        break

# Print the best set of features
print(features_lr)

['andrena', 'osmia', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MinOfLowerTRange', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']


In [13]:
# features_lr = ['andrena', 'osmia', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MinOfLowerTRange', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']

In [14]:
# Define the linear regression model
rf = RandomForestRegressor()

# Define the initial set of features to consider
features_rf = X.columns.to_list()

# Define the minimum MAE threshold
min_mae = np.inf

# Iterate over the features and remove the one that results in the lowest MAE
while len(features_rf) > 1:
    mae_values = []
    for feature in features_rf:
        temp_features = features_rf.copy()
        temp_features.remove(feature)
        rf.fit(X[temp_features], y)
        y_pred = rf.predict(X[temp_features])
        mae = mean_absolute_error(y, y_pred)
        mae_values.append(mae)
    best_feature_index = np.argmin(mae_values)
    best_feature_mae = mae_values[best_feature_index]
    if best_feature_mae < min_mae:
        min_mae = best_feature_mae
        features_rf.pop(best_feature_index)
    else:
        break

# Print the best set of features
print(features_rf)

['honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange', 'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']


In [15]:
# featrures_rf = ['honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange', 'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']

In [16]:
# Define the linear regression model
lgbm = LGBMRegressor()

# Define the initial set of features to consider
features_lgbm = X.columns.to_list()

# Define the minimum MAE threshold
min_mae = np.inf

# Iterate over the features and remove the one that results in the lowest MAE
while len(features_lgbm) > 1:
    mae_values = []
    for feature in features_lgbm:
        temp_features = features_lgbm.copy()
        temp_features.remove(feature)
        lgbm.fit(X[temp_features], y)
        y_pred = lgbm.predict(X[temp_features])
        mae = mean_absolute_error(y, y_pred)
        mae_values.append(mae)
    best_feature_index = np.argmin(mae_values)
    best_feature_mae = mae_values[best_feature_index]
    if best_feature_mae < min_mae:
        min_mae = best_feature_mae
        features_lgbm.pop(best_feature_index)
    else:
        break

# Print the best set of features
print(features_lgbm)

['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']


In [17]:
# features_lgbm = ['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']

In [18]:
# Define the linear regression model
xgb = XGBRegressor()

# Define the initial set of features to consider
features_xgb = X.columns.to_list()

# Define the minimum MAE threshold
min_mae = np.inf

# Iterate over the features and remove the one that results in the lowest MAE
while len(features_xgb) > 1:
    mae_values = []
    for feature in features_xgb:
        temp_features = features_xgb.copy()
        temp_features.remove(feature)
        xgb.fit(X[temp_features], y)
        y_pred = xgb.predict(X[temp_features])
        mae = mean_absolute_error(y, y_pred)
        mae_values.append(mae)
    best_feature_index = np.argmin(mae_values)
    best_feature_mae = mae_values[best_feature_index]
    if best_feature_mae < min_mae:
        min_mae = best_feature_mae
        features_xgb.pop(best_feature_index)
    else:
        break

# Print the best set of features
print(features_xgb)

['clonesize', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']


In [19]:
# features_xgb = ['clonesize', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']

In [20]:
# Define the linear regression model
gbr = GradientBoostingRegressor()

# Define the initial set of features to consider
features_gbr = X.columns.to_list()

# Define the minimum MAE threshold
min_mae = np.inf

# Iterate over the features and remove the one that results in the lowest MAE
while len(features_gbr) > 1:
    mae_values = []
    for feature in features_gbr:
        temp_features = features_gbr.copy()
        temp_features.remove(feature)
        gbr.fit(X[temp_features], y)
        y_pred = gbr.predict(X[temp_features])
        mae = mean_absolute_error(y, y_pred)
        mae_values.append(mae)
    best_feature_index = np.argmin(mae_values)
    best_feature_mae = mae_values[best_feature_index]
    if best_feature_mae < min_mae:
        min_mae = best_feature_mae
        features_gbr.pop(best_feature_index)
    else:
        break

# Print the best set of features
print(features_gbr)

['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'AverageOfUpperTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange', 'RainingDays', 'fruitset', 'fruitmass', 'seeds']


In [21]:
# features_gbr = ['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'AverageOfUpperTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange', 'RainingDays', 'fruitset', 'fruitmass', 'seeds']

# Base Model

In [22]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)

results = []

results.append(-cross_val_score(lr, X[features_lr], y, cv=cv, scoring='neg_mean_absolute_error'))

print(f'MAE: {np.mean(results).round(2)}')

MAE: 359.79


In [23]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)

results = []

results.append(-cross_val_score(rf, X[features_rf], y, cv=cv, scoring='neg_mean_absolute_error'))

print(f'MAE: {np.mean(results).round(2)}')

MAE: 356.07


In [24]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)

results = []

results.append(-cross_val_score(lgbm, X[features_lgbm], y, cv=cv, scoring='neg_mean_absolute_error'))

print(f'MAE: {np.mean(results).round(2)}')

MAE: 343.2


In [25]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)

results = []

results.append(-cross_val_score(xgb, X[features_xgb], y, cv=cv, scoring='neg_mean_absolute_error'))

print(f'MAE: {np.mean(results).round(2)}')

MAE: 356.63


In [26]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)

results = []

results.append(-cross_val_score(gbr, X[features_gbr], y, cv=cv, scoring='neg_mean_absolute_error'))

print(f'MAE: {np.mean(results).round(2)}')

MAE: 343.72


In [29]:
def rfr(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 1000, step=100)
    max_depth = trial.suggest_int('max_depth', 2, 30, step=2)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_float('max_features', 0.1, 1.0)
    
    model = RandomForestRegressor(n_estimators,
                                   max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   min_samples_leaf=min_samples_leaf,
                                   max_features=max_features)

    cv_scores = (-cross_val_score(model, X[features_rf], y, scoring='neg_mean_absolute_error', cv=cv))
    
    return np.mean(cv_scores)

def lgbm(trial):
    params = {
        'objective': 'regression',
        'metric': 'mse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 2, 30),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'min_gain_to_split': trial.suggest_loguniform('min_gain_to_split', 0.1, 5),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'random_state': 42,
        'verbose_eval': -1
    }
    
    model = LGBMRegressor(**params)
    
    model.fit(X, y)
    
    cv_scores = (-cross_val_score(model, X[features_lgbm], y, scoring='neg_mean_absolute_error', cv=cv))
    
    return np.mean(cv_scores)

def xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    subsample = trial.suggest_float('subsample', 0.1, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 1.0)
    
    model = XGBRegressor(n_estimators=n_estimators,
                             learning_rate=learning_rate,
                             max_depth=max_depth,
                             subsample=subsample,
                             colsample_bytree=colsample_bytree)
    
    model.fit(X, y)
    
    cv_scores = (-cross_val_score(model, X[features_xgb], y, scoring='neg_mean_absolute_error', cv=cv))
    
    return np.mean(cv_scores)

def gbr(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    model = GradientBoostingRegressor(n_estimators=n_estimators,
                                       learning_rate=learning_rate,
                                       max_depth=max_depth,
                                       min_samples_split=min_samples_split,
                                       min_samples_leaf=min_samples_leaf)
    
    model.fit(X, y)

    cv_scores = (-cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv))
    
    return np.mean(cv_scores)

In [30]:
study = optuna.create_study(direction='minimize')
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(rfr, n_trials=100)
study.best_params

C:\Users\Anes3\AppData\Roaming\Python\Python37\site-packages\optuna\distributions.py:686: UserWarning: The distribution is specified by [50, 1000] and step=100, but the range is not divisible by `step`. It will be replaced by [50, 950].
  low=low, old_high=old_high, high=high, step=step
C:\Users\Anes3\AppData\Roaming\Python\Python37\site-packages\optuna\distributions.py:686: UserWarning: The distribution is specified by [50, 1000] and step=100, but the range is not divisible by `step`. It will be replaced by [50, 950].
  low=low, old_high=old_high, high=high, step=step
C:\Users\Anes3\AppData\Roaming\Python\Python37\site-packages\optuna\distributions.py:686: UserWarning: The distribution is specified by [50, 1000] and step=100, but the range is not divisible by `step`. It will be replaced by [50, 950].
  low=low, old_high=old_high, high=high, step=step
C:\Users\Anes3\AppData\Roaming\Python\Python37\site-packages\optuna\distributions.py:686: UserWarning: The distribution is specified by 

{'n_estimators': 650,
 'max_depth': 10,
 'min_samples_split': 2,
 'min_samples_leaf': 6,
 'max_features': 0.6793496843486911}

In [31]:
study = optuna.create_study(direction='minimize')
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(lgbm, n_trials=100)
study.best_params

C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This 

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] feature_fraction is set=0.9935767228816483, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9935767228816483
[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=49 will be ignored. Current value: min_data_in_leaf=62
[LightGBM] [Warning] min_gain_to_split is set=0.887570575742724, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.887570575742724
[LightGBM] [Warning] bagging_fraction is set=0.5857400496519772, subsample=0.5876103104668567 will be ignored. Current value: bagging_fraction=0.5857400496519772
[LightGBM] [Warning] lambda_l1 is set=1.8658759965833105e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8658759965833105e-06
[LightGBM] [Warning] lambda_l2 is set=3.576325602080422e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.576325602080422e-08
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored

C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This 

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] feature_fraction is set=0.5807807592460386, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5807807592460386
[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=94 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] min_gain_to_split is set=2.9990055252205905, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.9990055252205905
[LightGBM] [Warning] bagging_fraction is set=0.6587330460588493, subsample=0.22951591571733682 will be ignored. Current value: bagging_fraction=0.6587330460588493
[LightGBM] [Warning] lambda_l1 is set=9.484252109957047e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.484252109957047e-06
[LightGBM] [Warning] lambda_l2 is set=0.005300670153214077, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.005300670153214077
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored.

C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This 

...

C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This 

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] feature_fraction is set=0.895931766436695, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.895931766436695
[LightGBM] [Warning] min_data_in_leaf is set=89, min_child_samples=17 will be ignored. Current value: min_data_in_leaf=89
[LightGBM] [Warning] min_gain_to_split is set=2.772879412798818, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.772879412798818
[LightGBM] [Warning] bagging_fraction is set=0.8427972538385234, subsample=0.8878752681244833 will be ignored. Current value: bagging_fraction=0.8427972538385234
[LightGBM] [Warning] lambda_l1 is set=0.01723049712919166, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.01723049712919166
[LightGBM] [Warning] lambda_l2 is set=0.0002382933420723581, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0002382933420723581
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Curren

C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
C:\Users\Anes3\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This 

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] feature_fraction is set=0.8129194961277204, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8129194961277204
[LightGBM] [Warning] min_data_in_leaf is set=61, min_child_samples=21 will be ignored. Current value: min_data_in_leaf=61
[LightGBM] [Warning] min_gain_to_split is set=1.0670845396517585, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.0670845396517585
[LightGBM] [Warning] bagging_fraction is set=0.749530765114653, subsample=0.9167214756082805 will be ignored. Current value: bagging_fraction=0.749530765114653
[LightGBM] [Warning] lambda_l1 is set=1.450173887942995, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.450173887942995
[LightGBM] [Warning] lambda_l2 is set=0.0017426604494200245, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0017426604494200245
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current 

{'num_leaves': 35,
 'learning_rate': 0.053281428574019164,
 'subsample': 0.8227003740190313,
 'feature_fraction': 0.8926700487141083,
 'bagging_fraction': 0.8595952907959205,
 'bagging_freq': 3,
 'min_child_samples': 9,
 'max_depth': 19,
 'lambda_l1': 0.049071635678460276,
 'lambda_l2': 0.0004209335796870315,
 'min_gain_to_split': 3.8331744352841954,
 'min_data_in_leaf': 55}

In [32]:
study = optuna.create_study(direction='minimize')
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(xgb, n_trials=100)
study.best_params

{'n_estimators': 544,
 'learning_rate': 0.011903538912680096,
 'max_depth': 7,
 'subsample': 0.3698694686676771,
 'colsample_bytree': 0.8939475744391724}

In [33]:
study = optuna.create_study(direction='minimize')
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(gbr, n_trials=100)
study.best_params

{'n_estimators': 379,
 'learning_rate': 0.015698218031068755,
 'max_depth': 6,
 'min_samples_split': 2,
 'min_samples_leaf': 9}

In [45]:
rfr_params = {
    'n_estimators': 650,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 6,
    'max_features': 0.6793496843486911
}

lgbm_params = {
    'num_leaves': 35,
    'learning_rate': 0.053281428574019164,
    'subsample': 0.8227003740190313,
    'feature_fraction': 0.8926700487141083,
    'bagging_fraction': 0.8595952907959205,
    'bagging_freq': 3,
    'min_child_samples': 9,
    'max_depth': 19,
    'lambda_l1': 0.049071635678460276,
    'lambda_l2': 0.0004209335796870315,
    'min_gain_to_split': 3.8331744352841954,
    'min_data_in_leaf': 55,
    'verbose': -1
}

xgb_params = {
    'n_estimators': 544,
    'learning_rate': 0.011903538912680096,
    'max_depth': 7,
    'subsample': 0.3698694686676771,
    'colsample_bytree': 0.8939475744391724
}

gbr_params = {
    'n_estimators': 379,
    'learning_rate': 0.015698218031068755,
    'max_depth': 6,
    'min_samples_split': 2,
    'min_samples_leaf': 9
}

In [46]:
rfr_model = RandomForestRegressor(**rfr_params)
lgbm_model = LGBMRegressor(**lgbm_params)
xgb_model = XGBRegressor(**xgb_params)
gbr_model = GradientBoostingRegressor(**gbr_params)

In [36]:
models = {
    'rf': rfr_model,
    'lgbm': lgbm_model,
    'xgb': xgb_model,
    'gbr': gbr_model
}

# Model Evaluation

In [37]:
results_ensemble_models = {}

for name, model in models.items():
    res=[]
    res.append(-cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv))
    results_ensemble_models[name] = res

[LightGBM] [Warning] feature_fraction is set=0.8926700487141083, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8926700487141083
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=9 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] min_gain_to_split is set=3.8331744352841954, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.8331744352841954
[LightGBM] [Warning] lambda_l1 is set=0.049071635678460276, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.049071635678460276
[LightGBM] [Warning] bagging_fraction is set=0.8595952907959205, subsample=0.8227003740190313 will be ignored. Current value: bagging_fraction=0.8595952907959205
[LightGBM] [Warning] lambda_l2 is set=0.0004209335796870315, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0004209335796870315
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] fea

In [38]:
for name, result in results_ensemble_models.items():
    print("----------\n" + name)
    print(np.mean(result))
    print(np.std(result))

----------
rf
338.01927497625996
10.751880096941267
----------
lgbm
342.18665615122717
10.246407351087269
----------
xgb
339.84309810449065
10.354003047740202
----------
gbr
340.89322828400753
9.703808571418268


In [47]:
final_model = VotingRegressor([('rfr', rfr_model),
                               ('lgbm', lgbm_model),
                               ('xgb', xgb_model),
                               ('gbr', gbr_model)],
                              weights=[.3, .1, .3, .2])

In [48]:
results_ensemble = []
    
results_ensemble.append(-cross_val_score(final_model, X, y, scoring='neg_mean_absolute_error', cv=cv))

print(np.mean(results_ensemble))

[LightGBM] [Warning] feature_fraction is set=0.8926700487141083, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8926700487141083
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=9 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] min_gain_to_split is set=3.8331744352841954, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.8331744352841954
[LightGBM] [Warning] lambda_l1 is set=0.049071635678460276, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.049071635678460276
[LightGBM] [Warning] bagging_fraction is set=0.8595952907959205, subsample=0.8227003740190313 will be ignored. Current value: bagging_fraction=0.8595952907959205
[LightGBM] [Warning] lambda_l2 is set=0.0004209335796870315, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0004209335796870315
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] fea

# Training

In [49]:
final_model.fit(X, y)

[LightGBM] [Warning] feature_fraction is set=0.8926700487141083, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8926700487141083
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=9 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] min_gain_to_split is set=3.8331744352841954, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.8331744352841954
[LightGBM] [Warning] lambda_l1 is set=0.049071635678460276, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.049071635678460276
[LightGBM] [Warning] bagging_fraction is set=0.8595952907959205, subsample=0.8227003740190313 will be ignored. Current value: bagging_fraction=0.8595952907959205
[LightGBM] [Warning] lambda_l2 is set=0.0004209335796870315, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0004209335796870315
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


VotingRegressor(estimators=[('rfr',
                             RandomForestRegressor(max_depth=10,
                                                   max_features=0.6793496843486911,
                                                   min_samples_leaf=6,
                                                   n_estimators=650)),
                            ('lgbm',
                             LGBMRegressor(bagging_fraction=0.8595952907959205,
                                           bagging_freq=3,
                                           feature_fraction=0.8926700487141083,
                                           lambda_l1=0.049071635678460276,
                                           lambda_l2=0.0004209335796870315,
                                           learning_rate=0.053281428574019164,
                                           max...
                                          n_estimators=544, n_jobs=None,
                                          num_parallel_tree=None

# Prediction

In [50]:
y_pred = final_model.predict(test_transformed)

# Submission

In [55]:
submission = pd.DataFrame(pd.concat([pd.Series(test.index, name='id'), pd.Series(y_pred, name='yield')], axis=1))

In [56]:
submission.to_csv('final_day_bagging.csv', index=False)